<h2 style="margin:0;"><strong>Consuming an XML File</strong></h2>
<h5 style="margin:0;"><em><strong>Hit the Net Hockey Store</strong></em></h5>
<h4>Populate a django database with an existing database from an xml file</h4>

<p><strong>Describing the problem</strong><br>I am building a webshop for a retailer of Ice Hockey Equipment. His stock depends on the stock and prices of another business he works closely with. This business has provided him with an XML file that has all their products on it.<br> Firstly, we want to dump this data into our own database. From there we will connect the databases using an API to create serialized signals when an item is out of stock.</p>

<p><strong><em>This notebook will modify the current xml file and transform it to a django fixture, matching our django database models.</em></strong><br> I took the liberty of using an online xml converter to convert the provided xml file to a json file before starting the notebook</p>


In [90]:
# import our libraries
import json
import pandas as pd 


<h3>Step 1: Prepare our existing data</h3>

In [81]:
# read our json file
file = pd.read_json("stockconvert.json")

,sku,qty,name,ean,price,recommended_retail_price,brand,description,shortdescription,category,...,maat,doelgroep,flex,breedte,kleur,merk,modelcode,niveau,side,images
0,377704019002,0,CCM RIBCOR TRIGGER 4 PRO Stick SR 85 19R,191520007010,170.0,"249,95",CCM,<h4>CCM RIBCOR TRIGGER 4 PRO Stick SR 85 19R k...,,[IJshockey],...,,Senior,85,0,0,CCM,HSRIB7SR85,Professioneel,Rechts,{'image': ['https://dealers.sportimex.com/b2b-...
1,200226002080,1,ROCES UFS PRO PLATE KIT M12 White UK 8 (41-42 EU),8020187608020,17.5,"35,00",ROCES,<h4>ROCES UFS PRO PLATE KIT M12 White UK 8 (41...,<p>Deze Roces M12 Pro plate is geschikt voor a...,[Skate-onderdelen],...,41-42,0,0,0,Wit,ROCES,200226,0,0,{'image': ['https://dealers.sportimex.com/b2b-...
2,200226002060,2,ROCES UFS PRO PLATE KIT M12 White UK 6 (39-40 EU),8020187640051,17.5,"35,00",ROCES,<h4>ROCES UFS PRO PLATE KIT M12 White UK 6 (39...,<p>Deze Roces M12 Pro plate is geschikt voor a...,[Skate-onderdelen],...,39-40,0,0,0,Wit,ROCES,200226,0,0,{'image': ['https://dealers.sportimex.com/b2b-...
3,200226002095,1,ROCES UFS PRO PLATE KIT M12 White UK 9.5 (43-4...,8020187608037,17.5,"35,00",ROCES,<h4>ROCES UFS PRO PLATE KIT M12 White UK 9.5 (...,<p>Deze Roces M12 Pro plate is geschikt voor a...,[Skate-onderdelen],...,43-44,0,0,0,Wit,ROCES,200226,0,0,{'image': ['https://dealers.sportimex.com/b2b-...
4,200226014080,2,ROCES UFS PRO PLATE KIT M12 Black UK 8 (41-42 EU),8020187591018,17.5,"35,00",ROCES,<h4>ROCES UFS PRO PLATE KIT M12 Black UK 8 (41...,,[Skate-onderdelen],...,41-42,0,0,0,Zwart,ROCES,200226,0,0,{'image': ['https://dealers.sportimex.com/b2b-...


In [82]:
# drop columns that I don't want (description is a generated text based on the other business's information and 'merk' is in the database twice, I don't want double data in my database)
description_dropped = file.drop("description", axis='columns')
merk_dropped = description_dropped.drop("merk", axis='columns')

# for this example I also dropped these columns but I may need them again later
cat_dropped = merk_dropped.drop("categories", axis='columns')
img_dropped = cat_dropped.drop("images", axis='columns')

,sku,qty,name,ean,price,recommended_retail_price,brand,shortdescription,category,curve,maat,doelgroep,flex,breedte,kleur,modelcode,niveau,side
0,377704019002,0,CCM RIBCOR TRIGGER 4 PRO Stick SR 85 19R,191520007010,170.0,"249,95",CCM,,[IJshockey],P19,,Senior,85,0,0,HSRIB7SR85,Professioneel,Rechts
1,200226002080,1,ROCES UFS PRO PLATE KIT M12 White UK 8 (41-42 EU),8020187608020,17.5,"35,00",ROCES,<p>Deze Roces M12 Pro plate is geschikt voor a...,[Skate-onderdelen],0,41-42,0,0,0,Wit,200226,0,0
2,200226002060,2,ROCES UFS PRO PLATE KIT M12 White UK 6 (39-40 EU),8020187640051,17.5,"35,00",ROCES,<p>Deze Roces M12 Pro plate is geschikt voor a...,[Skate-onderdelen],0,39-40,0,0,0,Wit,200226,0,0
3,200226002095,1,ROCES UFS PRO PLATE KIT M12 White UK 9.5 (43-4...,8020187608037,17.5,"35,00",ROCES,<p>Deze Roces M12 Pro plate is geschikt voor a...,[Skate-onderdelen],0,43-44,0,0,0,Wit,200226,0,0
4,200226014080,2,ROCES UFS PRO PLATE KIT M12 Black UK 8 (41-42 EU),8020187591018,17.5,"35,00",ROCES,,[Skate-onderdelen],0,41-42,0,0,0,Zwart,200226,0,0


In [83]:
#Showing the different data types in each column
merk_dropped.dtypes

sku                          object
qty                           int64
name                         object
ean                          object
price                       float64
recommended_retail_price     object
brand                        object
shortdescription             object
category                     object
categories                   object
curve                        object
maat                         object
doelgroep                    object
flex                         object
breedte                      object
kleur                        object
modelcode                    object
niveau                       object
side                         object
images                       object
dtype: object

<h3>Step 2: Creating new columns of data and adding it to the dataframe</h3>

<h4>2.1 : Defining the django model</h4>

In [84]:
# creating a list as long as the dataframe that defines the django model to create from the fixture

new_column = []

for i in range(0, len(merk_dropped)):
    new_column.append("products.product")

In [85]:
# Adding the created column to our dataframe
merk_dropped['model'] = new_column

<h4>2.2 : Defining the object primary key</h4>

In [86]:
# creating a list as long as the dataframe that defines the django model's primary key

# empty list
pk_column = []

# setting a base pk for my loop
pk = 0

#looping through the dataframe length and exponentially add 1 to the pk
for i in range(0, len(merk_dropped)):
    pk += 1
    pk_column.append(pk)

In [87]:
# Adding the created column to our dataframe
merk_dropped['pk'] = pk_column

<h4>2.3 : Rearranging the dataframe columns</h4>

In [88]:
# get the existing list of columns
merk_dropped.columns

Index(['sku', 'qty', 'name', 'ean', 'price', 'recommended_retail_price',
       'brand', 'shortdescription', 'category', 'categories', 'curve', 'maat',
       'doelgroep', 'flex', 'breedte', 'kleur', 'modelcode', 'niveau', 'side',
       'images', 'model', 'pk'],
      dtype='object')

In [89]:
# copy the list from the above output and reorder the columns to fit my preference using following line of code
    ## notice the double brackets, they are necessary as otherwise a KeyError is passed
merk_dropped = merk_dropped[['pk', 'model', 'sku', 'qty', 'name', 'ean', 'price', 'recommended_retail_price', 'brand', 'shortdescription', 'category', 'categories', 'curve', 'maat', 'doelgroep', 'flex', 'breedte', 'kleur', 'modelcode', 'niveau', 'side', 'images']]

<h3>Step 3 : Writing our dataframe to a .json fixture for Django</h3>

In [77]:
#create JSON file 
json_file = img_dropped.to_json(orient='records') 

#export JSON file
with open('product_fixture.json', 'w') as f:
    f.write(json_file)

<h2>Conclusion</h2>
<p>This was a very easy way to prepare the exisiting data and populate my new database without having to manually enter the objects one by one (would be crazy work for 39k unique products)</p>

<h4>Next Steps</h4>
<ul>
    <li><h5>Adjust my databse models in django</h5></li>
    <li><h5>Create a separate fixture for translating the Category objects and hooking these up to this dataframe with foreignkeys</h5></li>
    <li><h5>Populate the database with the transformed data</h5></li>
    <li><h5>Create an API using 'django Restful API' to comunicate live stock levels</h5></li>
</ul>